In [1]:
import pandas as pd
unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
users = pd.read_csv('ml-1m/users.dat', sep='::', header=None, names=unames, engine='python', encoding='latin-1')
rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('ml-1m/ratings.dat', sep='::', header=None, names=rnames,  engine='python', encoding='latin-1')
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_csv('ml-1m/movies.dat', sep='::', header=None, names=mnames,  engine='python', encoding='latin-1')
data = pd.merge(pd.merge(ratings, users), movies)

# Exercises: 

**MovieLens database**

1- Filter  films that have received at least 250 ratings. **Use only these films in the rest of exercices**

In [2]:
def ex1():
    ratings = data.groupby('title').size() 
    ratings = ratings[ratings>= 250].sort_values(ascending=False)
    
    return ratings
     

2- Obtain the mean ratings for each movie grouped by gender that have at least 250 ratings. 

In [3]:
def ex2():
    subset_ratings = ex1()
    i_drop = data[~data['title'].isin(subset_ratings.index)].index
    data_2 = data.drop(i_drop)
    
    ratings = data_2[["title", "rating", "gender"]].groupby(["gender","title"]).mean()
    ratings = pd.merge(ratings.loc["F"], ratings.loc["M"], on="title", suffixes=('_F', '_M'))

    return ratings

3- Show films more valued by women.

In [4]:
def ex3():
    ratings = ex2()
    ratings = ratings.sort_values(by='rating_F', ascending= False)['rating_F']
    
    return ratings

4- Now we wonder which movies are rated more differently between men and women. Which films have more different rating and are more highly valued by women? 

In [5]:
def ex4():
    ratings = ex2()
    ratings["diff"]=ratings["rating_M"]-ratings["rating_F"]
    ratings = ratings.sort_values(by='diff')
    return ratings

5- And which films have more different rating and are more highly valued by men? 

In [6]:
def ex5():
    ratings = ex2()
    ratings["diff"]=ratings["rating_M"]-ratings["rating_F"]
    ratings = ratings.sort_values(by='diff', ascending=False)
    return ratings

6- What are the films that have generated the most discordant ratings, regardless of gender?

In [7]:
def ex6():
    subset_ratings = ex1()
    i_drop = data[~data['title'].isin(subset_ratings.index)].index
    data_2 = data.drop(i_drop)
    
    ratings = data_2[["title", "rating"]].groupby(["title"]).std()
    ratings = ratings.sort_values(by='rating', ascending=False)
    return ratings

7- What is the highest rated movie in average?

In [8]:
def ex7():
    subset_ratings = ex1()
    i_drop = data[~data['title'].isin(subset_ratings.index)].index
    data_2 = data.drop(i_drop)
    
    ratings = data_2[["title", "rating"]].groupby(["title"]).mean()
    ratings = ratings.sort_values(by='rating', ascending=False)
    ratings.drop(ratings.index[1:],inplace=True)
    return ratings

*** From here use all the ratings ***

8- Calculate the average rating of each user. 

In [9]:
def ex8():
    ratings = data[["user_id", "rating"]].groupby(["user_id"]).mean()
    return ratings

9- Define a function called  <b>top_movies</b> that given a df, an user an a number n it returns what movies have the top n rank for this user.

def top_movies(data,user,n)


In [10]:
def top_movies(data_,usr,n_rows = 10):
    ratings = data_[data_.user_id == usr][["title","rating"]].sort_values(by="rating",ascending=False)[:n_rows]
    return ratings
def ex9():
    return top_movies(data,1,30)


** Data from CSV**

10- Read data from csv file: `ma-ba.csv`. Count the number of times `Barça` wins `Madrid` and compute the stadistics of % win, % lose and % draw.

In [11]:
def ex10():
    unames = ["N","Fecha", "Lugar", "Partidos", "Resultado", "Competición"]
    results = pd.read_csv('ma-ba.csv',header=None,skiprows=1, names = unames, engine='python').set_index("N")
    results[['equipo_1', 'equipo_2']] = results['Partidos'].str.split('-',expand=True)
    results["equipo_1"] = results["equipo_1"].replace({'Madrid FC ': 'Madrid','Barcelona ':'Barça','R. Madrid ':'Madrid','R.Madrid  ':'Madrid' })
    results["equipo_2"] = results["equipo_2"].replace({' Madrid FC': 'Madrid',' Barcelona':'Barça',' R. Madrid':'Madrid',' R.Madrid ':'Madrid' })
    results[['result_1', 'result_2']] = results['Resultado'].str.split('-',expand=True)
    results[["Barça_win", "Barça_lose", "Barça_draw"]] = results[['equipo_1', 'equipo_2', 'result_1', 'result_2']].apply(
        lambda d: [False, False, True] if d[2]==d[3] 
        else([True, False, False] if (d[2]>d[3] and d[0]=="Barça") or
                                     (d[2]<d[3] and d[1]=="Barça")
            else [False, True, False]), axis=1, result_type="expand")
        
    wins = sum(results["Barça_win"])
    wins_perc = wins/len(results)
    loses = sum(results["Barça_lose"])
    loses_perc = loses/len(results)
    draws = sum(results["Barça_draw"])
    draws_perc = draws/len(results)    
    results = pd.DataFrame(columns=["wins", "loses", "draws", "perc_win", "perc_lose", "perc_draw"])
    results.loc["Barcelona"]= [wins, loses, draws, wins_perc, loses_perc, draws_perc]
    results.loc["R. Madrid"]= [loses, wins, draws, loses_perc, wins_perc, draws_perc]
    
    return results

In [12]:
n = 11
for i in range(1,n):
    try:
        df = globals()["ex"+str(i)]()
        df.to_csv(str(i)+"_claudia.csv",header=False)
    except Exception as e: 
        print(i,repr(e))
        open(str(i)+".csv","a").close()


10 UnicodeDecodeError('utf-8', b'  N\xb0,   Fecha,       (Lugar),         Partidos,      resultado,   Competici\xf3n\n1,13-05-1902,(Madrid)   ,Madrid FC - Barcelona,1-3,Copa del Rey (1/2)\n2,13-05-1906,(Barcelona),Barcelona - Madrid FC,5-2,Partido amistoso\n3,01-11-1913,(Barcelona),Barcelona - Madrid FC,7-0,Partido amistoso\n4,02-11-1913,(Barcelona),Barcelona - Madrid FC,1-0,Partido amistoso\n5,06-01-1914,(Madrid)   ,Madrid FC - Barcelona,2-2,Partido amistoso\n6,10-01-1914,(Madrid)   ,Madrid FC - Barcelona,0-2,Partido amistoso\n7,05-03-1916,(Barcelona),Barcelona - Madrid FC,3-0,Partido amistoso\n8,26-03-1916,(Barcelona),Barcelona - Madrid FC,2-1,Copa del Rey (1/2)\n9,02-04-1916,(Madrid)   ,Madrid FC - Barcelona,4-1,Copa del Rey (1/2)\n10,13-04-1916,(Madrid)   ,Madrid FC - Barcelona,6-6,Copa del Rey (1/2)\n11,15-04-1916,(Madrid)   ,Madrid FC - Barcelona,4-2,Copa del Rey (1/2)\n12,07-03-1916,(Barcelona),Barcelona - Madrid FC,0-0,Partido amistoso\n13,01-11-1917,(Barcelona),Barcelona - Mad